### Библиотеки

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import re
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict

#### Подгружаем данные

In [2]:
url_train="https://drive.google.com/u/0/uc?id=1RhXDaw02wlVtvUggA9C81i78vDTM8u14&export=download"
train = pd.read_csv(url_train) # мой подготовленный датасет для обучения модели
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

#### сначало тест

In [3]:
#оставляем 'mileage'
test.drop(['modelDate', 
            'vehicleConfiguration',
            'description',
            'Комплектация',
            'ПТС',
            'Таможня', 
            'Владение', 
            'id',
            'name'], inplace = True, axis = 1)

#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10,
'лифтбек' : 11,
'родстер' : 12,
'кабриолет' : 13}

test.loc[(test["bodyType"] == "седан 2 дв.") , 'bodyType'] = "седан"
test.loc[(test["bodyType"] == "внедорожник 5 дв.") , 'bodyType'] = "внедорожник"
test.loc[(test["bodyType"] == "хэтчбек 5 дв.") , 'bodyType'] = "хэтчбек"
test.loc[(test["bodyType"] == "универсал 5 дв.") , 'bodyType'] = "универсал"
test.loc[(test["bodyType"] == "компактвэн") , 'bodyType'] = "микроавтобус"
test.loc[(test["bodyType"] == "хэтчбек 3 дв.") , 'bodyType'] = "хэтчбек"
#Заменяем
test.bodyType = test.bodyType.astype(np.str)
#Заменяем
test['bodyType'] = test['bodyType'].map(dict_bodyType)

#В столбце "Руль" есть пропуски, убираем.
print(test.Руль.value_counts())
print(test.Руль.isna().sum())
fs = test[test.Руль.isnull()]

test.loc[(test['brand'] == "BMW") & (test.Руль.isnull()) , 'Руль'] = "Левый"
test.loc[(test.Руль.isnull()) , 'Руль'] = "Правый"

#Нашел статью на хабре, пробуем https://habr.com/ru/post/456294/  , а то в прошлый раз все словарями делал. 
LE = LabelEncoder()

LE.fit(test['color'])
test['Color_new'] = LE.transform(test['color'])
test.drop(['color'],axis = 1, inplace = True)

LE.fit(test['vehicleTransmission'])
test['vehicleTransmission_new'] = LE.transform(test['vehicleTransmission'])
test.drop(['vehicleTransmission'],axis = 1, inplace = True)

LE.fit(test['Привод'])
test['Привод_new'] = LE.transform(test['Привод'])
test.drop(['Привод'],axis = 1, inplace = True)

LE = LabelEncoder()
#LE.fit(test['Руль']) Выдает ошибку, может кто то поймет как с ней бороться
#test['Руль_new'] = LE.transform(test['Руль'])
#test.drop(['Руль'],axis = 1, inplace = True)
LE.fit(test['Состояние'])
test['Состояние_new'] = LE.transform(test['Состояние'])
test.drop(['Состояние'],axis = 1, inplace = True)

LE.fit(test['fuelType'])
test['fuelType_new'] = LE.transform(test['fuelType'])
test.drop(['fuelType'],axis = 1, inplace = True)


LE.fit(test['brand'])
test['brand_new'] = LE.transform(test['brand'])
test.drop(['brand'],axis = 1, inplace = True)


#Чистим от л.с.
test['enginePower'] = test['enginePower'].apply(lambda x: float(str(x).split()[0]))



test.dropna(subset = ['engineDisplacement'],inplace = True)
test.loc[(test["engineDisplacement"] == "6.0+") , 'engineDisplacement'] = "7.0"
test.loc[(test["engineDisplacement"] == "undefined LTR") , 'engineDisplacement'] = "3.0"
test['engineDisplacement']=test['engineDisplacement'].apply(lambda x: str(str(x).split()[0]))



test[test.numberOfDoors.isna()]
test.dropna(subset = ['numberOfDoors'],inplace = True)

test.numberOfDoors = test.numberOfDoors.astype(np.int32)

test['Владельцы']=test['Владельцы'].apply(lambda x: int(str(x).split()[0]))
test['Владельцы'].astype(np.int)



#Дропаем оставшиеся признаки

#test.drop(['Электростеклоподъемники', 'Усилитель руля', 'Климат-контроль','Салон','Диски','marketPrice'], inplace = True, axis = 1)


#переводим engineDisplacement из obj в float
test.engineDisplacement = test.engineDisplacement.astype(np.float)

#забыл про руль

dict_rul={"Левый": 1, "Правый": 2}
test['Руль'] = test['Руль'].map(dict_rul)

#Оставим одни значения BMW
#test = test[test['brand'].isin(['BMW'])]
#test.drop(['brand'], inplace = True, axis = 1)


#Переводим в int productionDate и mileage, enginePower, Владельцы 
test.productionDate = test.productionDate.astype(np.int64)
test.mileage = test.mileage.astype(np.int64)
test.enginePower = test.enginePower.astype(np.int32)
test['Владельцы'] = test['Владельцы'].astype(np.int32)





Левый    3837
Name: Руль, dtype: int64
0


### теперь Train

In [4]:
train.drop(['autoCatalogUrl', 'status', 'closingReason','canonicalUrl','avitoId','Аудиосистема','Фары', 'description'], inplace = True, axis = 1)

train.dropna(subset = ['bodyType'],inplace = True)

#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10}

#Заменяем
train['bodyType'] = train['bodyType'].map(dict_bodyType)

#В столбце "Руль" есть пропуски, убираем.
print(train.Руль.value_counts())
print(train.Руль.isna().sum())
fs = train[train.Руль.isnull()]

train.loc[(train['brand'] == "BMW") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Renault") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Volkswagen") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Peugeot") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Chevrolet") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Audi") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Opel") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Lexus") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Skoda") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Mercedes-Benz") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train.Руль.isnull()) , 'Руль'] = "Правый"

#Нашел статью на хабре, пробуем https://habr.com/ru/post/456294/  , а то в прошлый раз все словарями делал. 
LE = LabelEncoder()
LE.fit(train['color'])
train['Color_new'] = LE.transform(train['color'])
train.drop(['color'],axis = 1, inplace = True)
LE.fit(train['vehicleTransmission'])
train['vehicleTransmission_new'] = LE.transform(train['vehicleTransmission'])
train.drop(['vehicleTransmission'],axis = 1, inplace = True)
LE.fit(train['Привод'])
train['Привод_new'] = LE.transform(train['Привод'])
train.drop(['Привод'],axis = 1, inplace = True)
LE = LabelEncoder()
#LE.fit(train['Руль']) Выдает ошибку, может кто то поймет как с ней бороться
#train['Руль_new'] = LE.transform(train['Руль'])
#train.drop(['Руль'],axis = 1, inplace = True)
LE.fit(train['Состояние'])
train['Состояние_new'] = LE.transform(train['Состояние'])
train.drop(['Состояние'],axis = 1, inplace = True)
LE.fit(train['name'])
train['name_new'] = LE.transform(train['name'])
train.drop(['name'],axis = 1, inplace = True)
LE.fit(train['fuelType'])
train['fuelType_new'] = LE.transform(train['fuelType'])
train.drop(['fuelType'],axis = 1, inplace = True)
LE.fit(train['typeOfTrade'])
train['typeOfTrade_new'] = LE.transform(train['typeOfTrade'])
train.drop(['typeOfTrade'],axis = 1, inplace = True)


#Чистим от л.с.
train['enginePower'] = train['enginePower'].apply(lambda x: float(str(x).split()[0]))

train.dropna(subset = ['engineDisplacement'],inplace = True)

train.loc[(train["engineDisplacement"] == "6.0+") , 'engineDisplacement'] = "7.0"

train.enginePower = train.enginePower.astype(np.int32)


train[train.numberOfDoors.isna()]
train.dropna(subset = ['numberOfDoors'],inplace = True)

train.numberOfDoors = train.numberOfDoors.astype(np.int32)

train.loc[(train["Владельцы"] == "4+") , 'Владельцы'] = 4

train.Владельцы = train.Владельцы.astype(np.int32)


LE.fit(train['brand'])
train['brand_new'] = LE.transform(train['brand'])
train.drop(['brand'],axis = 1, inplace = True)

#Дропаем оставшиеся признаки

train.drop(['Электростеклоподъемники', 'Усилитель руля', 'Климат-контроль','Салон','Диски','marketPrice'], inplace = True, axis = 1)


### разбираемся с brand
# all_brands = train.brand.value_counts().index
# top_brands = list(all_brands)[:top_brands_count]
# brands_to_throw_away = list(set(all_brands) - set(top_brands))
# train.loc[train['brand'].isin(brands_to_throw_away), 
#              'brand'] = 'other'


# brands_df = pd.get_dummies(train.brand, drop_first=True)  
# train = pd.concat([train, brands_df],axis = 1)

# #дропаем первоначальную колонку brand
# train.drop(['brand'], inplace = True, axis = 1)

#переводим engineDisplacement из obj в float
train.engineDisplacement = train.engineDisplacement.astype(np.float)

#забыл про руль

dict_rul={"Левый": 1, "Правый": 2}
train['Руль'] = train['Руль'].map(dict_rul)

#Оставим одни значения BMW
# train = train[train['brand'].isin(['BMW'])]
# train.drop(['brand'], inplace = True, axis = 1)

# Дропаем 
train.drop(['typeOfTrade_new'], inplace = True, axis = 1)

#дропаем name
train.drop(['name_new'], inplace = True, axis = 1)


### заполним в тесте поле brand_new "девятками"

test.loc[(test["brand_new"] == 0) , 'brand_new'] = 9

Левый     36017
Правый     1145
Name: Руль, dtype: int64
194


### добавляем в модель и в трейн два новых признака по примеру датасета Максима

In [5]:
#налоговые ставки

test['Налог']=test.enginePower.apply(lambda x: x*12 if x<=100 
                                           else x*25 if x>100 and x<=125
                                           else x*35 if x>125 and x<=150
                                           else x*45 if x>150 and x<=175
                                           else x*55 if x>175 and x<=200
                                           else x*65 if x>200 and x<=225
                                           else x*75 if x>225 and x<=250
                                           else x*150)
train['Налог']=train.enginePower.apply(lambda x: x*12 if x<=100 
                                           else x*25 if x>100 and x<=125
                                           else x*35 if x>125 and x<=150
                                           else x*45 if x>150 and x<=175
                                           else x*55 if x>175 and x<=200
                                           else x*65 if x>200 and x<=225
                                           else x*75 if x>225 and x<=250
                                           else x*150)

In [6]:
#Сколько лет автомобилю
test['Время эксплуатации'] = 2020 - test['productionDate']
train['Время эксплуатации'] = 2020 - train['productionDate']

In [7]:
#средний пробег
# test['Средний пробег'] =test['mileage'].astype(float) / test['Время эксплуатации'].astype(float)
# train['Средний пробег'] =train['mileage'].astype(float) / train['Время эксплуатации'].astype(float)
# test['Средний пробег']=test['Средний пробег'].round()
# train['Средний пробег']=train['Средний пробег'].round()


In [8]:
# train.drop(['Налог','engineDisplacement', 'Время эксплуатации','Руль','Color_new','bodyType'], inplace = True, axis = 1)
# test.drop(['Налог','engineDisplacement', 'Время эксплуатации','Руль','Color_new','bodyType'], inplace = True, axis = 1)
train.drop(['engineDisplacement'], inplace = True, axis = 1)
test.drop(['engineDisplacement'], inplace = True, axis = 1)


### Модель

In [9]:
from sklearn.preprocessing import RobustScaler

X = train.drop(['itemPrice'], axis = 1)
y = train['itemPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)
# # нормализуем X_train
# transformer = RobustScaler()
# X_train = transformer.fit_transform(X_train)

# # normalising X_test
# transformer = RobustScaler()
# X_test = transformer.fit_transform(X_test)

# # normalising X_sub
# transformer = RobustScaler()
# X_sub = transformer.fit_transform(X_sub)

In [10]:
#Случайный лес
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict

RANDOM_SEED = 42
forest = RandomForestRegressor(n_estimators=10, max_features=9,  random_state=RANDOM_SEED) #max_depth = 15, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100, verbose = 100)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred1 = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')

print_regression_metrics(y_test, y_pred)

In [11]:
# Используем обученную модель для предсказания цены авто в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
#y_pred = forest.predict(X_test)

X_sub = test

predict_submission = forest.predict(X_sub)
predict_submission
sample_submission['price'] = predict_submission
sample_submission.price = sample_submission.price.astype(np.int32)
sample_submission.to_csv(f'submission_v.csv', index=False)
sample_submission.head(10)

,id,price
0,0,1672499
1,1,2147000
2,2,767200
3,3,2330600
4,4,6270900
5,5,1660500
6,6,1009300
7,7,676500
8,8,1418500
9,9,2651000
